In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
#Load Datasets
initial_data = pd.read_csv('../01_Training_data/training_data_labels.csv')

fit_data = pd.read_csv('../03_Testing_data/testing_data.csv')

In [3]:
#Select columns
training_data = initial_data[["basin","population","construction_year","extraction_type","management","payment_type","water_quality","quantity","source"]]
#training_data.head(3)
training_data["train"] = 1

fit_data_set = fit_data[["basin","population","construction_year","extraction_type","management","payment_type","water_quality","quantity","source"]]
fit_data_set["train"] = 0
#fit_data_set.head(3)

/home/chamodyadias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/chamodyadias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [4]:
combined = pd.concat([training_data,fit_data_set])
combined.shape

(74250, 10)

In [5]:
#Encode Categorical data
label_encoder = preprocessing.LabelEncoder()
combined['basin'] = label_encoder.fit_transform(combined['basin'])
combined['extraction_type'] = label_encoder.fit_transform(combined['extraction_type'])
combined['management'] = label_encoder.fit_transform(combined['management'])
combined['payment_type'] = label_encoder.fit_transform(combined['payment_type'])
combined['water_quality'] = label_encoder.fit_transform(combined['water_quality'])
combined['quantity'] = label_encoder.fit_transform(combined['quantity'])
combined['source'] = label_encoder.fit_transform(combined['source'])
#combined['status_group'] = label_encoder.fit_transform(combined['status_group'])
combined.head(8)

,basin,population,construction_year,extraction_type,management,payment_type,water_quality,quantity,source,train
0,1,109,1999,3,7,0,6,1,8,1
1,4,280,2010,3,11,2,6,2,5,1
2,5,250,2009,3,7,5,6,1,0,1
3,7,58,1986,14,7,2,6,0,3,1
4,4,0,0,3,1,2,6,3,5,1
5,5,1,2009,14,7,5,4,1,4,1
6,0,0,0,15,7,2,6,1,3,1
7,3,0,0,8,11,6,3,1,7,1


In [6]:
train_df = combined[combined["train"]==1]
train_df.drop(["train"],axis=1,inplace=True)
test_df = combined[combined["train"]==0]
test_df.drop(["train"],axis=1,inplace=True)
train_df.shape
test_df.shape

/home/chamodyadias/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(14850, 9)

In [7]:
train_df["status_group"] = initial_data[["status_group"]]
train_df.shape

/home/chamodyadias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(59400, 10)

In [8]:
train_df.head(20)

,basin,population,construction_year,extraction_type,management,payment_type,water_quality,quantity,source,status_group
0,1,109,1999,3,7,0,6,1,8,functional
1,4,280,2010,3,11,2,6,2,5,functional
2,5,250,2009,3,7,5,6,1,0,functional
3,7,58,1986,14,7,2,6,0,3,non functional
4,4,0,0,3,1,2,6,3,5,functional
5,5,1,2009,14,7,5,4,1,4,functional
6,0,0,0,15,7,2,6,1,3,non functional
7,3,0,0,8,11,6,3,1,7,non functional
8,3,0,0,4,7,2,4,3,3,non functional
9,4,0,0,8,7,2,6,1,7,functional


In [13]:
train_df["status_group"] = train_df["status_group"].replace(to_replace ="functional", 
                 value =0)

/home/chamodyadias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
train_df["status_group"] = train_df["status_group"].replace(to_replace ="non functional", 
                 value =1)

/home/chamodyadias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
train_df["status_group"] = train_df["status_group"].replace(to_replace ="functional needs repair", 
                 value =2)

/home/chamodyadias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
train_df.head(20)

,basin,population,construction_year,extraction_type,management,payment_type,water_quality,quantity,source,status_group
0,1,109,1999,3,7,0,6,1,8,0
1,4,280,2010,3,11,2,6,2,5,0
2,5,250,2009,3,7,5,6,1,0,0
3,7,58,1986,14,7,2,6,0,3,1
4,4,0,0,3,1,2,6,3,5,0
5,5,1,2009,14,7,5,4,1,4,0
6,0,0,0,15,7,2,6,1,3,1
7,3,0,0,8,11,6,3,1,7,1
8,3,0,0,4,7,2,4,3,3,1
9,4,0,0,8,7,2,6,1,7,0


In [17]:
train, test = train_test_split(train_df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

38016 train examples
9504 validation examples
11880 test examples


In [18]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('status_group')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [19]:
feature_columns = []

# numeric cols
for header in ["basin","population","construction_year","extraction_type","management","payment_type","water_quality","quantity","source"]:
  feature_columns.append(feature_column.numeric_column(header))

In [20]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [21]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [26]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

Epoch 1/50
1188/1188 [==============================] - 4s 4ms/step - loss: 2.7438 - acc: 0.4680 - val_loss: 3.5599 - val_acc: 0.3897
Epoch 2/50
1188/1188 [==============================] - 3s 3ms/step - loss: 1.8978 - acc: 0.4956 - val_loss: 1.6719 - val_acc: 0.5662
Epoch 3/50
1188/1188 [==============================] - 3s 3ms/step - loss: 1.3927 - acc: 0.5078 - val_loss: 0.6907 - val_acc: 0.5712
Epoch 4/50
1188/1188 [==============================] - 3s 3ms/step - loss: 1.0515 - acc: 0.5174 - val_loss: 1.0246 - val_acc: 0.4366
Epoch 5/50
1188/1188 [==============================] - 3s 3ms/step - loss: 0.9520 - acc: 0.5214 - val_loss: 1.1550 - val_acc: 0.5725
Epoch 6/50
1188/1188 [==============================] - 3s 3ms/step - loss: 0.8796 - acc: 0.5217 - val_loss: 1.8176 - val_acc: 0.5547
Epoch 7/50
1188/1188 [==============================] - 3s 3ms/step - loss: 0.8149 - acc: 0.5308 - val_loss: 0.6667 - val_acc: 0.5207
Epoch 8/50
1188/1188 [==============================] - 3s 3ms

In [27]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

372/372 [==============================] - 1s 2ms/step - loss: 0.5019 - acc: 0.5570
Accuracy 0.5569865
